Data Prep and Feature Engineering

Author: Ansh Nenwani

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import swifter
import warnings
%matplotlib inline
sns.set_theme(style='darkgrid')
warnings.filterwarnings(action='once')

## Data Preparation

In [2]:
# Reading in all the necessary data
races_df = pd.read_csv('Data/races.csv')
results_df = pd.read_csv('Data/results.csv')

In [3]:
results_df = results_df.merge(races_df[['year','round', 'raceId', 'circuitId']], left_on='raceId', right_on='raceId', how='left', suffixes=['','_drop'])
results_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId
0,1,18,1,1,22,1,1,1,1,10.0,...,1:34:50.616,5690616,39,2,1:27.452,218.300,1,2008,1,1
1,2,18,2,2,3,5,2,2,2,8.0,...,+5.478,5696094,41,3,1:27.739,217.586,1,2008,1,1
2,3,18,3,3,7,7,3,3,3,6.0,...,+8.163,5698779,41,5,1:28.090,216.719,1,2008,1,1
3,4,18,4,4,5,11,4,4,4,5.0,...,+17.181,5707797,58,7,1:28.603,215.464,1,2008,1,1
4,5,18,5,1,23,3,5,5,5,4.0,...,+18.014,5708630,43,1,1:27.418,218.385,1,2008,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26075,26081,1110,817,213,3,19,16,16,16,0.0,...,+1:43.071,5053521,25,15,1:50.994,227.169,1,2023,12,13
26076,26082,1110,858,3,2,18,17,17,17,0.0,...,+1:44.476,5054926,37,9,1:50.486,228.213,1,2023,12,13
26077,26083,1110,807,210,27,0,18,18,18,0.0,...,+1:50.450,5060900,26,4,1:49.907,229.415,1,2023,12,13
26078,26084,1110,832,6,55,4,\N,R,19,0.0,...,\N,\N,9,19,1:53.138,222.864,130,2023,12,13


In [4]:
drop_columns = []
for column in results_df.columns:
    if '_extra' in column:
        drop_columns.append(column)
results_df.drop(columns=drop_columns, inplace=True)
results_df.drop(columns=['number','position','positionText', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed'], inplace=True)

## Feature Engineering

In [5]:
results_df['top_3'] = results_df.positionOrder < 4
results_df['top_3_label'] = np.where(results_df['top_3'] == True, 'yes', 'no')

In [6]:
def top3_finishes(row, df):
    top_3_count = df[(df.year == row.year) & (df['round'] < row['round']) & (df.driverId == row.driverId)].top_3.sum()
    top_3_freq = top_3_count / df[(df.year == row.year) & (df.driverId == row.driverId) & (df['round'] < row['round'])].driverId.count()
    return top_3_freq * 100
    

In [7]:
results_df['top3_driver_season_percentage'] = results_df.swifter.apply(top3_finishes, axis=1, args=(results_df,))
results_df.info()

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

/tmp/ipykernel_6973/3880751005.py:3: RuntimeWarning: invalid value encountered in scalar divide
  top_3_freq = top_3_count / df[(df.year == row.year) & (df.driverId == row.driverId) & (df['round'] < row['round'])].driverId.count()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26080 entries, 0 to 26079
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   resultId                       26080 non-null  int64  
 1   raceId                         26080 non-null  int64  
 2   driverId                       26080 non-null  int64  
 3   constructorId                  26080 non-null  int64  
 4   grid                           26080 non-null  int64  
 5   positionOrder                  26080 non-null  int64  
 6   points                         26080 non-null  float64
 7   statusId                       26080 non-null  int64  
 8   year                           26080 non-null  int64  
 9   round                          26080 non-null  int64  
 10  circuitId                      26080 non-null  int64  
 11  top_3                          26080 non-null  bool   
 12  top_3_label                    26080 non-null 

In [8]:
def avg_finish_position_season(row, df):
    return df[(df.driverId == row.driverId) & (df.year == df.year) & (df['round'] < row['round'])].positionOrder.mean()

In [9]:
results_df['driver_avg_finish_pos_season'] = results_df.swifter.apply(avg_finish_position_season, axis=1, args=(results_df,))

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

In [10]:
def constructor_top_3(row, df):
    top_3_count = df[(df['year'] == row.year) & (df.constructorId == row.constructorId) & (df['round'] < row['round'])].top_3.sum()
    top_3_freq = top_3_count / df[(df.year == row.year) & (df.constructorId == row.constructorId) & (df['round'] < row['round'])].driverId.count()
    return top_3_freq * 100

In [11]:
results_df['Constructor_Top3_Percent'] = results_df.swifter.apply(constructor_top_3, axis=1, args=(results_df,))

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

/tmp/ipykernel_6973/3025212643.py:3: RuntimeWarning: invalid value encountered in scalar divide
  top_3_freq = top_3_count / df[(df.year == row.year) & (df.constructorId == row.constructorId) & (df['round'] < row['round'])].driverId.count()


In [12]:
def percent_top_3_at_circuit(row, df):
    return (df[(df.circuitId == row.circuitId) & (df.driverId == row.driverId) & (df['year'] < row['year'])].top_3.sum() / df[(df.circuitId == row.circuitId) & (df.driverId == row.driverId)].circuitId.count()) * 100

In [13]:
results_df['Top_3_at_circuit'] = results_df.swifter.apply(percent_top_3_at_circuit, axis=1, args=(results_df,))

/home/alexsearle/anaconda3/envs/testing/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Pandas Apply:   0%|          | 0/26080 [00:00<?, ?it/s]

In [14]:
results_df.fillna(0, inplace=True)

In [15]:
max_round = pd.DataFrame(results_df.groupby(['year'], as_index=False)['round'].max(), columns=['year', 'round'])
max_round['year'] = max_round['year'] + 1

In [16]:
last_race_stats = results_df.merge(max_round, on=['year'], how='left', suffixes=['', '_max_last_szn'])

In [17]:
last_race_stats.dropna(inplace=True)
lag_df = results_df.copy()
lag_df['year'] = lag_df['year'] + 1
last_race_stats = last_race_stats.merge(lag_df[['top3_driver_season_percentage', 'driver_avg_finish_pos_season', 'Constructor_Top3_Percent','year', 'round', 'driverId']], left_on=['year', 'round_max_last_szn', 'driverId'],right_on=['year', 'round', 'driverId'], how='left', suffixes=('', '_lag'))

In [18]:
last_race_stats = last_race_stats.sort_values(['year', 'round'], ascending=True)

In [19]:
results_df = last_race_stats.copy()
results_df.fillna(0, inplace=True)
results_df.drop(columns=['resultId', 'raceId', 'constructorId'], inplace=True)
results_df[['positionOrder', 'grid']].astype('int')
results_df['year'] = pd.to_datetime(results_df['year'], format='%Y')

In [20]:
results_df.to_csv('Data/Modeling_v1.csv')